In [1]:
import pandas as pd
pop = pd.read_csv('../data/mock_sysselsatte/population_2019_30000.csv')
comp = pd.read_csv('../data/mock_sysselsatte/companies_2019_30000.csv')

### Nedfiltrering, en essensiell kompetanse

In [3]:
# De fra 20 tom. 30 år i datasettet
pop_20_30 = pop[    
    (pop['age'] >= 20)  
    &  (pop['age'] <= 30)    
]
# Se på aldrene som nå eksisterer i datasettet, sortert
sorted(  pd.unique( pop_20_30['age'] )  )

[20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]

### Lag grupperingsvariabler etter en inndeling av alder

In [5]:
pop2 = pop.copy()
# Litt juks, ta bort siste tegnet i alderen
pop2['age_group'] = pop2['age'].astype(str).str[:-1]
# De som er tomme på alder nå, får gruppen "0"
pop2['age_group'] = pop2['age_group'].replace('', '0')
# Concatenate en 0 på slutten
pop2['age_group'] = pop2['age_group'] + '0' 
# Se på alder og aldersgruppe
pop2[['age', 'age_group']].sample(10)

,age,age_group
6959,21,20
20305,32,30
18037,18,10
11492,66,60
2442,15,10
10887,45,40
1321,6,00
10995,62,60
8379,31,30
26088,62,60


### Litt mer sofistikert eller "generell"

In [7]:
# Ta en ny kopi av datasettet
pop3 = pop.copy()

In [8]:
# Se på datatypene til datasettet
pop3.dtypes

Unnamed: 0        int64
id                int64
work_id         float64
sex              object
age               int64
year_birth        int64
work_percent    float64
dtype: object

In [10]:
list(range(0, 120, 10))

[0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110]

In [9]:
last = ''
# Alle tall fra 0 til, men ikke med 120, med ett "step" på 10
for ar in range(0, 120, 10):
    print(ar)
    # Hopp over første loopen, men gå inn her om forrige var 0
    if last or last == 0:
        # Om forrige var 0 prøver vi å få med de som var 0 år
        if last == 0:
            pop3.loc[ ( pop3['age'] >= last) & ( pop3['age'] <= ar ), 'age_group'] = f'{last} - {ar}' 
            print(last, ar)
            print(f"Forrige år var {last}, og vi har da spesielle regler for å få med de som er 0 år")
        # Dette er da for de gruppene, hvor første år ikke er 0
        else:
            # Legg merke til at likhetstegnet er borte fra den første parantesen
            pop3.loc[ ( pop3['age'] > last) & ( pop3['age'] <= ar ), 'age_group'] = f'{last + 1} - {ar}' 
            print(last + 1, ar)
            print(f"Forrige år var ikke 0, da starter vi 1 over {last}")
    last = ar

0
10
0 10
Forrige år var 0, og vi har da spesielle regler for å få med de som er 0 år
20
11 20
Forrige år var ikke 0, da starter vi 1 over 10
30
21 30
Forrige år var ikke 0, da starter vi 1 over 20
40
31 40
Forrige år var ikke 0, da starter vi 1 over 30
50
41 50
Forrige år var ikke 0, da starter vi 1 over 40
60
51 60
Forrige år var ikke 0, da starter vi 1 over 50
70
61 70
Forrige år var ikke 0, da starter vi 1 over 60
80
71 80
Forrige år var ikke 0, da starter vi 1 over 70
90
81 90
Forrige år var ikke 0, da starter vi 1 over 80
100
91 100
Forrige år var ikke 0, da starter vi 1 over 90
110
101 110
Forrige år var ikke 0, da starter vi 1 over 100


In [11]:
pop3[['age', 'age_group']].sample(10)

,age,age_group
2628,14,11 - 20
19502,34,31 - 40
27868,68,61 - 70
15893,5,0 - 10
12697,47,41 - 50
21448,39,31 - 40
1031,9,0 - 10
9635,64,61 - 70
27773,78,71 - 80
20344,44,41 - 50


### Sett en verdi i en kolonne basert på en verdi i en annen kolonne, med ett .loc-søk

In [12]:
pop3.loc[pop3['age'] == 0, 'status'] = "nyfodt"

In [13]:
pop3

,Unnamed: 0,id,work_id,sex,age,year_birth,work_percent,age_group,status
0,0,29101946112,NaN,Menn,0,2019,NaN,0 - 10,nyfodt
1,1,26101930288,NaN,Menn,0,2019,NaN,0 - 10,nyfodt
2,2,22091952214,NaN,Menn,0,2019,NaN,0 - 10,nyfodt
3,3,4071997222,NaN,Menn,0,2019,NaN,0 - 10,nyfodt
4,4,7101970642,NaN,Menn,0,2019,NaN,0 - 10,nyfodt
...,...,...,...,...,...,...,...,...,...
29995,29995,15072660901,NaN,Kvinner,93,1926,NaN,91 - 100,NaN
29996,29996,23012604128,NaN,Kvinner,93,1926,NaN,91 - 100,NaN
29997,29997,14102931680,NaN,Kvinner,90,1929,NaN,81 - 90,NaN
29998,29998,29062932768,NaN,Kvinner,90,1929,NaN,81 - 90,NaN


### Det samme, bare at vi looper over en dict med "nivåer"

In [14]:
statuser = {
    13 : 'tenaring',
    18 : 'voksen',
    30 : 'middelaldrende',
    60 : 'eldre'
}

De senere vil skrive over de første, noe som kan være ett problem om vi ikke kan garantere at keysene kommer i rekkefølgen...

In [15]:
for k, i in statuser.items():
    print(k, i)
    pop3.loc[pop3['age'] >= k, 'status'] = i

13 tenaring
18 voksen
30 middelaldrende
60 eldre


In [16]:
pop3[['age','status']].sample(10)

,age,status
14453,72,eldre
4454,38,middelaldrende
27769,67,eldre
2736,15,tenaring
7819,23,voksen
4322,39,middelaldrende
11408,48,middelaldrende
25331,62,eldre
20940,34,middelaldrende
20213,28,voksen


In [ ]:
# Kan vi stole på rekkefølgen i en dict? Eller er det en annen datatype vi heller bør satse på da?
from collections import OrderedDict
help(OrderedDict)